# Model: Mistral 7B Instruct v0.1

### Installs

In [ ]:
!pip install optimum
!pip install git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 974.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git (to revision 72958fcd3c98a7afdc61f953aa58c544ebda2f79) to /tmp/pip-req-build-sx76fed1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-sx76fed1
  Running command git rev-parse -q --verify 'sha^72958fcd3c98a7afdc61f953aa58c544ebda2f79'
  Running command git fetch -q https://github.com/huggingface/t

### Imports

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

### Configurations

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

# prompt related
prompt = "Tell me about AI"
prompt_template=f'''<s>[INST] {prompt} [/INST]
'''


### Utility Methods

In [ ]:
# return model and tokenizer
def get_model():
  model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

  tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
  return model, tokenizer

# get prediction
def get_prediction(model, tokenizer):
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
  output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
  return tokenizer.decode(output[0])

# get prediction #2 (second way)
def get_prediction2(model, tokenizer):
  pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1)
  return pipe(prompt_template)[0]['generated_text']


### Download Model, tokenizer

In [ ]:
model, tokenizer = get_model()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Execution (Method1)

In [ ]:
import time

start = time.time()
predicted_text = get_prediction(model, tokenizer)
end = time.time()

# show time of execution per iteration
print(f"Predicted Text: {predicted_text}")
print(f"Time taken: {(end-start)*10**3:.03f}ms")


Predicted Text: <s><s>[INST] Tell me about AI [/INST]
AI stands for Artificial Intelligence. It is a branch of computer science that focuses on creating intelligent machines that can perform tasks that would normally require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems use algorithms, machine learning, and other techniques to analyze data and make predictions or decisions based on that data. There are many different types of AI, including narrow or weak AI, which is designed to perform a specific task, and general or strong AI, which is designed to be capable of any intellectual task that a human can perform. AI has many applications, including in healthcare, finance, transportation, and entertainment.</s>
Time taken: 5857.020ms


### Execution (Method2)

In [ ]:
import time

start = time.time()
predicted_text = get_prediction2(model, tokenizer)
end = time.time()

# show time of execution per iteration
print(f"Predicted Text: {predicted_text}")
print(f"Time taken: {(end-start)*10**3:.03f}ms")

Predicted Text: <s>[INST] Tell me about AI [/INST]
AI, or artificial intelligence, is a branch of computer science that focuses on the development of intelligent machines that can perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, and making decisions. AI systems use algorithms, machine learning techniques, and neural networks to process information and learn from their experiences, enabling them to improve their performance over time. There are many different types of AI, including narrow or weak AI, which is designed to perform a specific task, and general or strong AI, which has the ability to reason, think abstractly, and understand the world in the same way as humans do. Some common applications of AI include virtual assistants, self-driving cars, speech recognition, and recommendation engines.
Time taken: 7080.424ms
